<a href="https://colab.research.google.com/github/AnastasiaZhivilo/AnastasiaZhivilo/blob/main/data_prep/Bands_research.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import ee
import folium
import google.colab.drive as drive
import geemap
import os

In [ ]:
ee.Authenticate()
ee.Initialize(project='cs88-2-capstone')
print("Earth Engine authenticated and initialized.")

In [ ]:
drive.mount('/content/drive/')

In [ ]:

drive_mount_path = '/content/drive'

# 1. Define the central point
center_lon = 146.9561
center_lat = -34.7024
center_point = ee.Geometry.Point(center_lon, center_lat)

# 2. Define the size (radius) for the buffer in meters
buffer_radius_meters = 2000  # 2 kilometers

# 3. Create a geodesic buffer (circle) around the point
circle_geometry = center_point.buffer(buffer_radius_meters)

# 4. Get the Bounding Box (Rectangle) coordinates from the buffered geometry
# .bounds() returns an ee.Geometry.Rectangle
bounding_box_geom = circle_geometry.bounds()

bbox_coords = bounding_box_geom.getInfo()['coordinates'][0]

# 5. Set up a folder name for this bounding box
folder_name = f"BBox_{center_lon:.4f}_{center_lat:.4f}".replace('.', '_').replace('-', '_')

filepath = os.path.join(drive_mount_path, 'MyDrive', 'Capstone', folder_name)

In [ ]:
def add_indices(image):
    """
    Calculates NDVI and SAVI, and applies a smoothing filter to NDVI.
    The resulting image will have the new bands: 'NDVI' (smoothed) and 'SAVI'.
    """
    # 1. Calculate NDVI: (B8 - B4) / (B8 + B4)
    ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI_raw')

    # 2. Smooth NDVI using a 3x3 median filter to reduce noise while preserving edges
    # The kernel is set to a radius of 1 pixel, resulting in a 3x3 window (2*1 + 1)
    ndvi_smoothed = ndvi.focal_median(radius=1, kernelType='square', units='pixels').rename('NDVI')

    # 3. Calculate SAVI: Soil Adjusted Vegetation Index (L=0.5)
    # SAVI = ((B8 - B4) / (B8 + B4 + L)) * (1 + L)
    L = ee.Number(0.5)
    savi = image.expression(
        '((NIR - RED) / (NIR + RED + L)) * (1 + L)', {
            'NIR': image.select('B8'),
            'RED': image.select('B4'),
            'L': L
        }).rename('SAVI')

    # --- 3. Normalized Difference Red-Edge (NDRE) ---
    # Use B8 and B5 to check chlorophyll in dense canopies
    ndre = image.normalizedDifference(['B8', 'B5']).rename('NDRE')

        # --- 4. Bare Soil Index (BSI) ---
    # Highlights bare soil using SWIR, Red, NIR, and Blue bands
    bsi = image.expression(
        '((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2))', {
        'B11': image.select('B11'),
        'B4': image.select('B4'),
        'B8': image.select('B8'),
        'B2': image.select('B2')
    }).rename('BSI')

    # Return the original image with the two new derived bands
    return image.addBands(ndvi_smoothed).addBands(savi).addBands(ndre).addBands(bsi).addBands(ndvi)


# TARGET BANDS now includes the two new indices (Smoothed NDVI and SAVI)

EXPORT_SCALE = 10  # 10 meters resolution

# Define different date ranges for separate GeoTIFF exports
# Adjusted for Southern Hemisphere (NSW Riverina Winter Crops)
DATE_RANGES = [
    # 1a. Canola Peak (Early Spring): Captures the bright yellow flowering stage
    {'start': '2024-09-01', 'end': '2024-09-30', 'name_suffix': 'CanolaPeak'},

    # 1b. Barley Senescence (Mid-Spring): Barley starts ripening, maximizing contrast with green wheat.
    {'start': '2024-10-01', 'end': '2024-10-31', 'name_suffix': 'BarleySenescence'},

    # 1c. Wheat Peak (Late Spring): Wheat is fully developed/filling, while barley is dry.
    {'start': '2024-11-01', 'end': '2024-11-30', 'name_suffix': 'WheatPeak'},

    # 2. Establishment/Early Growth (Winter): Sowing and Emergence stage
    {'start': '2024-05-01', 'end': '2024-07-31', 'name_suffix': 'Emergence'},

    # 3. Post-Harvest/Bare Earth (Summer): Fields are typically bare or harvested
    {'start': '2023-12-01', 'end': '2024-02-29', 'name_suffix': 'BareEarth'}
]

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image."""
  qa = image.select('QA60')
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11
  mask = (
      qa.bitwiseAnd(cloud_bit_mask).eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )
  # Scale the image data by dividing by 10000 (0-1 range)
  return image.updateMask(mask).divide(10000)

In [ ]:
start_date = '2024-12-01'
end_date = '2024-12-31'

# 1. Filter and process the ImageCollection
s2_collection = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    .filterBounds(bounding_box_geom)
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .map(mask_s2_clouds)
    # Add indices (NDVI, SAVI) and smooth NDVI
    .map(add_indices)
)


############# Percentiles #################
#2. Create the median composite and clip to the geometry
mean_composite = s2_collection.median().clip(bounding_box_geom)
#percentile_composite = s2_collection.reduce(ee.Reducer.percentile([5, 95])).clip(bounding_box_geom)

TARGET_BANDS = ['SAVI','NDVI_raw','NDVI','BSI']

# 3. Select only the required bands (now 6 total)
final_image = mean_composite.select(TARGET_BANDS)

########### NDVI std Dev ########
# # 2. Create the Standard Deviation composite for NDVI and clip to the geometry
# # Only select the NDVI band and apply the standard deviation reducer
# ndvi_std_dev = (
#     s2_collection
#     .select('B8')
#     .reduce(ee.Reducer.stdDev())
#     .rename('NDVI_stdDev')
#     .clip(bounding_box_geom)
# )

# # 3. Select only the required band(s)
# # Since you only want the Standard Deviation of NDVI, this is the final image.
# final_image = ndvi_std_dev.select(['NDVI_stdDev'])

######################################

visualization = {
    'min': 0.03,
    'max': 1,
}

final_image = mean_composite.select(['NDVI'])


In [ ]:


Map = geemap.Map(center=[center_lat, center_lon], zoom=12)
Map.addLayer(final_image, visualization, 'Median Sentinel-2 Image')
Map.zoom_to_bounds(bounding_box_geom.bounds().getInfo().get('coordinates')[0])
Map